<h1 align="center" style="font-size: 3.5em; text-decoration: underline;">✨ Tools Kit ✨</h1>
<p align="center">
<a href="https://schwallergroup.github.io/practical-programming-in-chemistry/"><img src="https://img.shields.io/badge/PPCHEM-3776AB.svg?style=for-the-badge&logo=python&logoColor=white" alt="PPCHEM"></a>
 </p>

## 📋 Project Context
    
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

## ⚔️ Challenge Faced

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.



import


In [29]:
%matplotlib inline
import tkinter as tk
from tkinter import messagebox, filedialog, Canvas, Entry, Button, PhotoImage
from rdkit.Chem import rdDepictor
import sys
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
from datetime import datetime


setup

In [30]:
sys.path.insert(0, "../src")
rdDepictor.SetPreferCoordGen(True)
set_matplotlib_formats('png', 'pdf')
plt.rcParams['font.family'] = 'Times New Roman'

## 🧪 Features

 ### 1. 🧬 Molecule Name
   - Input: Raw formula of the molecule
   - Output: SMILES of the molecule

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.
### 2. ⚖️ Molecular Weight
   - Input: SMILES of the molecule
   - Output: Corresponding molar mass in g/mol

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 3. 📈 Linear Regression
   - Input: Excel document (imported by pressing the Browse button)
   - Output: Linear regression graph with the R<sup>2</sup> value
        - Options: The graph allows customization of various parameters such as changing scales, axis labels, title, displaying maximum values, enabling gridlines, adjusting line types and colors, modifying grid and label axes, and background color.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 4. 📊 Graph Maker
   - Input: Excel document with x and y values arranged in two columns
   - Output: Graph plotting all values
      - Options: The graph allows customization of various parameters such as changing scales, axis labels, title, displaying maximum values, enabling gridlines, adjusting line types and colors, modifying grid and label axes, and background color.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 5. 🧮 Error Propagation
   - Input: Variables, their values, and their uncertainties.
   - Output: Mean value, its uncertainty, and the result can be copied as LaTeX code for easy inclusion in documents.

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.

### 6. 🔬 Show Molecule
   - Input: SMILES
   - Output: Molecular structure

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed dictum consectetur lectus, vitae tincidunt urna accumsan vel. Nunc in enim nulla. Vestibulum eget lorem nec nisi lobortis feugiat. Nullam lacinia pharetra lorem, vitae hendrerit dui fermentum nec. Sed non felis nec justo posuere interdum. Integer euismod magna nec dui vehicula, id fermentum orci hendrerit. Integer scelerisque nec urna vel lacinia. Vivamus convallis, dui a euismod lacinia, ex enim tincidunt magna, vitae suscipit lacus enim non nisi. Curabitur id justo justo. Nunc accumsan eget arcu at rhoncus. Etiam at sapien vel odio scelerisque fermentum. Sed tincidunt, justo a ultricies malesuada, sem justo consequat tellus, et ultricies elit arcu ac lorem. Ut auctor purus nec libero sodales, quis consectetur eros hendrerit. Vivamus varius semper neque, at tincidunt neque sollicitudin at.


import function


In [31]:
from project_ppchem_tools_kit.smiles_to_molar_mass import smiles_to_molar_mass
from project_ppchem_tools_kit.name_to_smiles import name_to_smiles
from project_ppchem_tools_kit.display_molecule import display_molecule
from project_ppchem_tools_kit.make_graph import make_graph
from project_ppchem_tools_kit.linear_regression import linear_regression
from project_ppchem_tools_kit.error_calculation_interface import error_calculation_interface
from project_ppchem_tools_kit.relative_to_assets import relative_to_assets
from project_ppchem_tools_kit.display_result import display_result
from project_ppchem_tools_kit.welcome_message import welcome_message

In [32]:
entry_input = None
selected_radio = None
window = None
x_label = None
y_label = None
title = None
result_text = None

In [33]:


def process_input(event=None):
    global selected_radio, x_label, y_label, title, result_text
    if selected_radio.get() == "1":
        smiles_molecule = name_to_smiles(entry_input.get().strip())
        if smiles_molecule is not None:
            result_text = f"The SMILES for the molecule '{entry_input.get().strip()}' is: {smiles_molecule}"
        else:
            result_text = f"The molecule '{entry_input.get().strip()}' was not found in the PubChem database."
    elif selected_radio.get() == "2":
        molar_mass = smiles_to_molar_mass(entry_input.get().strip())
        if molar_mass is not None:
            result_text = f"The molar mass of the molecule is: {molar_mass} g/mol"
        else:
            result_text = "Invalid SMILES or molecule not found."
    elif selected_radio.get() in ["3", "4"]:
        if selected_radio.get() == "3":
            make_graph(entry_input, x_label, y_label, title)
        else:
            linear_regression(entry_input, x_label, y_label, title)
        return 
    elif selected_radio.get() == "5":  
        display_molecule(entry_input)  
        return 
    elif selected_radio.get() == "6":
        error_calculation_interface()
    else:
        result_text = "Please select an input type."

    display_result(result_text)

In [34]:
def browse_excel_file():
    global entry_input
    filepath = filedialog.askopenfilename(title="Select Excel File", filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*")))
    if filepath:
        try:
            entry_input.delete(0, tk.END)  
            entry_input.insert(0, filepath)  
            print("Selected Excel file:", filepath)
            print("File path copied to clipboard.")
        except Exception as e:
            print("Error:", e)

def clear_input(): 
    """Clears the content of the input field in the GUI.
    """
    entry_input.delete(0, tk.END)

In [35]:
window = tk.Tk()  

x_axis_label = tk.StringVar()
y_axis_label = tk.StringVar()
graph_title = tk.StringVar()

window.title("Project Tools-Kit")
input_type = tk.IntVar() 
input_type.set(1)

window.geometry("1000x800")
window.configure(bg = "#1B262C")

canvas = Canvas(window, bg="#1B262C", height=800, width=1000, bd=0, highlightthickness=0, relief="ridge")
canvas.place(x=0, y=0)

canvas.create_text(12.0, 194.0, anchor="nw", text="Choose Input Type :", fill="#BBE1FA", font=("Times New Roman", 27, "bold"))

canvas.create_text(92.0, 378.0, anchor="nw",text="Input :", fill="#BBE1FA", font=("Times New Roman", 30, "bold"))
entry_input = Entry(bd=1, bg="#0F4C75", fg="#BBE1FA", highlightthickness=0, font=("Times New Roman", 25))
entry_input.place(x=272.5, y=368.0, width=605.0, height=74.0)

button_image_browse = PhotoImage(file=relative_to_assets("button_browse.png")) 
button_browse = Button(image=button_image_browse, borderwidth=0, highlightthickness=0, command=browse_excel_file, relief="flat")
button_browse.place(x=409.0, y=500.0, width=154.2904052734375, height=60.0)

button_image_process = PhotoImage(file=relative_to_assets("button_process.png"))
button_process = Button(image=button_image_process, borderwidth=0, highlightthickness=0, command=process_input, relief="flat")
button_process.place(x=394.5, y=620.0, width=182.0, height=73.0)

button_image_title = PhotoImage(file=relative_to_assets("button_title.png"))
button_title = Button(image=button_image_title, borderwidth=0, highlightthickness=0, command=welcome_message, relief="flat")
button_title.place(x=273.0, y=36.0, width=455.0, height=90.0)



logo_image = PhotoImage(file=relative_to_assets('EPFL_logo.png'))
logo_label = tk.Label(window, image=logo_image)
logo_label.place(x=10.0, y=10.0, width=150.0, height=60.0)


In [36]:
def clear_input(): 
    entry_input.delete(0, tk.END)

In [37]:
def on_radio_select(value):
    selected_radio.set(value)
    clear_input()

In [38]:
def create_radio_button(x, y, text, value):  
    radio_button = tk.Radiobutton(canvas, text=text, variable=selected_radio, value=value,
                                  command=lambda: on_radio_select(value),
                                  font=("Times New Roman", 20), bg="#1B262C")
    canvas.create_window(x, y, anchor="nw", window=radio_button)

In [39]:
radio_button_data = [(291, 194, "Molecule Name", "1"),(291, 252, "Excel Graph", "3"),(700, 194, "Linear Regression", "4"), (700, 252, "Show Molecule", "5"),(469, 252, "Error calculation", "6"),(469, 194, "Molecular weight", "2")]  
selected_radio = tk.StringVar(value="none")
for data in radio_button_data:
    create_radio_button(*data)

In [40]:
def update_clock():
    current_time = datetime.now().strftime('%H:%M:%S')
    current_date = datetime.now().strftime('%Y-%m-%d')
    clock_label.config(text=current_time)
    date_label.config(text=current_date)
    window.after(1000, update_clock)

date_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
date_label.place(x=890, y=10)

clock_label = tk.Label(window, text="", font=("Times New Roman", 20), bg="#1B262C", fg="#FFFFFF")
clock_label.place(x=890, y=40)
update_clock()

In [41]:
def on_closing():
    global window
    if messagebox.askokcancel("Quit", "Are you sure you want to quit?"):
        window.destroy()
window.protocol("WM_DELETE_WINDOW", on_closing)

''

In [42]:
window.bind("<Return>", process_input)

window.mainloop()